## Risk Parity Approach
---

A <font color=red>risk parity approach</font> of portfolio management focuses on balancing "risk allocation" among assets, instead of balancing the target return and the risk of the portfolio.

1. The mean-variance approach tends to produce an extremely skewed portfolio.
2. It is hard to obtain a reliable estimate of the expected return on any asset.
3. Many fund managers feel increasingly uncomfortable with the traditional asset allocation techniques since they were useless during such financial turmoil as the Global Financial Crisis.



In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as opt
Mu = np.array([1.0, 3.0, 1.5, 6.0, 4.5])
Stdev = np.array([5.0, 10.0, 7.5, 15.0, 11.0])
CorrMatrix = np.array([[1.00, 0.25, 0.18, 0.10, 0.25],
                       [0.25, 1.00, 0.36, 0.20, 0.20],
                       [0.18, 0.36, 1.00, 0.25, 0.36],
                       [0.10, 0.20, 0.25, 1.00, 0.45],
                       [0.25, 0.20, 0.36, 0.45, 1.00]])
Sigma = np.diag(Stdev) @ CorrMatrix @ np.diag(Stdev)
iota = np.ones(Mu.shape)
inv_Sigma = la.inv(Sigma)

### $1/N$ Portfolio
---

One trivial example of such portfolios that equalize the impact of each asset onto the total risk is

\begin{equation*}
 w_{n}^{1/N} = \frac1{N},\quad (n=1,\dots,N).
\end{equation*}

This is often refered to as the <font color=red>$1/N$ portfolio</font>.


In [2]:
Weight_1N = np.tile(1.0/Mu.shape[0], Mu.shape[0])

### Global Minimum Variance Portfolio
---

The <font color=red>global minimum variance portfolio</font> is the solution of the following optimization problem:

\begin{align*}
 \min_{w} &\quad w^{\intercal}\Sigma w \\
 \text{subject to} &\quad w^{\intercal}\iota = 1,
\end{align*}

that is,

\begin{equation*}
 w^{MV} = \frac1{\iota^{\intercal}\Sigma^{-1}\iota}\Sigma^{-1}\iota.
\end{equation*}

Note that $w^{MV}$ does not depend on the expected return vector $\mu$. Thus we do not need to estimate $\mu$.


In [3]:
Weight_MV = inv_Sigma @ iota / (iota @ inv_Sigma @ iota)

### Risk Parity Portfolio
---

A portfolio that satisfies

\begin{equation*}
 w_1\nabla_1\sigma(w) = \cdots = w_N\nabla_N\sigma(w),\quad
 \sigma(w)=\sqrt{w^{\intercal}\Sigma w},\quad
 \nabla_n\sigma(w) = \frac{\partial\sigma(w)}{\partial w_n},
\end{equation*}
is called the <font color=red>risk parity portfolio</font>.

The allocation weight of the risk parity portfolio $w^{RP}$ is the solution of the system of non-linear equations:

\begin{align*}
 \Sigma w^{RP} &= \frac{\kappa}{w^{RP}}
 = \begin{bmatrix}
 \displaystyle
 \frac{\kappa}{w_{1}^{RP}} \\ \vdots \\
 \displaystyle
 \frac{\kappa}{w_{N}^{RP}}
 \end{bmatrix}, \\
 \iota^{\intercal}w^{RP} &= 1,
\end{align*}

which has no closed-form solution, but can be solved numerically.


In [4]:
F = lambda v, Sigma: np.hstack((Sigma @ v[:-1] - v[-1]/v[:-1], v[:-1].sum() - 1.0))
Weight_RP = opt.root(F, np.hstack((Weight_1N, 0.0)), args=Sigma).x[:-1]

### Maximum Diversification Portfolio
---

The <font color=red>maximum diversification portfolio</font> is the solution of

\begin{equation*}
 \max_{w} \quad \frac{\sigma^{\intercal}w}{\sqrt{w^{\intercal}\Sigma w}}.
\end{equation*}

It is given by

\begin{equation*}
 w^{MD} = \frac1{\iota^{\intercal}\Sigma^{-1}\sigma}\Sigma^{-1}\sigma.
\end{equation*}


In [5]:
Weight_MD = inv_Sigma @ Stdev / (iota @ inv_Sigma @ Stdev)

In [6]:
np.set_printoptions(formatter={'float': '{:7.2f}'.format})
print(np.vstack((Weight_1N, Weight_MV, Weight_RP, Weight_MD))*100)

[[  20.00   20.00   20.00   20.00   20.00]
 [  69.77    3.79   23.33    2.98    0.14]
 [  36.52   16.78   21.39   11.35   13.96]
 [  42.81   16.10   19.22   12.61    9.26]]
